##   Imputaciones simples


Una opción es imputar por la media, pero altera el resultado

Otra opción es usar la estructura de correlación y un modelo de prección


In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.impute import SimpleImputer

In [2]:
data = pd.read_csv("base_datos_2008.csv")

In [6]:
#df = data.dropna(subset = ["AirTime", "Distance", "TaxiIn", "TaxiOut", "DepDelay", "ArrDelay", "CarrierDelay"])
df = data.sample(frac = 1).head(1000) #Selección aleatoria

X = df[["AirTime", "Distance", "TaxiIn", "TaxiOut", "DepDelay", "ArrDelay", "CarrierDelay"]]

In [7]:
X

,AirTime,Distance,TaxiIn,TaxiOut,DepDelay,ArrDelay,CarrierDelay
6418345,174.0,1041,3.0,44.0,-3.0,26.0,0.0
4739911,161.0,1197,4.0,11.0,14.0,5.0,NaN
477653,281.0,2586,5.0,23.0,-7.0,-18.0,NaN
21581,31.0,192,3.0,10.0,-2.0,-3.0,NaN
6702193,61.0,366,6.0,30.0,-2.0,13.0,NaN
...,...,...,...,...,...,...,...
5848324,194.0,1438,12.0,17.0,-8.0,5.0,NaN
292334,43.0,279,3.0,27.0,44.0,30.0,0.0
2594354,130.0,916,5.0,13.0,-3.0,-5.0,NaN
4037582,91.0,612,6.0,15.0,5.0,-3.0,NaN


In [9]:
imp = SimpleImputer(missing_values=np.nan, strategy="mean")
imp.fit(X)

SimpleImputer()

In [10]:
X2 = imp.fit_transform(X)

In [11]:
pd.DataFrame(X2)

,0,1,2,3,4,5,6
0,174.0,1041.0,3.0,44.0,-3.0,26.0,0.000000
1,161.0,1197.0,4.0,11.0,14.0,5.0,16.482759
2,281.0,2586.0,5.0,23.0,-7.0,-18.0,16.482759
3,31.0,192.0,3.0,10.0,-2.0,-3.0,16.482759
4,61.0,366.0,6.0,30.0,-2.0,13.0,16.482759
...,...,...,...,...,...,...,...
995,194.0,1438.0,12.0,17.0,-8.0,5.0,16.482759
996,43.0,279.0,3.0,27.0,44.0,30.0,0.000000
997,130.0,916.0,5.0,13.0,-3.0,-5.0,16.482759
998,91.0,612.0,6.0,15.0,5.0,-3.0,16.482759


In [12]:
X.corr()

,AirTime,Distance,TaxiIn,TaxiOut,DepDelay,ArrDelay,CarrierDelay
AirTime,1.000000,0.983051,0.086714,0.125606,0.012733,0.021304,0.007397
Distance,0.983051,1.000000,0.078774,0.113617,-0.000405,-0.015786,0.026169
TaxiIn,0.086714,0.078774,1.000000,0.025999,-0.020026,0.080423,-0.109572
TaxiOut,0.125606,0.113617,0.025999,1.000000,0.133065,0.348759,-0.100501
DepDelay,0.012733,-0.000405,-0.020026,0.133065,1.000000,0.933291,0.497568
ArrDelay,0.021304,-0.015786,0.080423,0.348759,0.933291,1.000000,0.453296
CarrierDelay,0.007397,0.026169,-0.109572,-0.100501,0.497568,0.453296,1.000000


In [14]:
X = X.dropna(subset = ["DepDelay", "ArrDelay"])
new_X = X[["DepDelay", "ArrDelay", "CarrierDelay"]]
new_X_train = new_X[np.isnan(X.CarrierDelay)==False]

regr = linear_model.LinearRegression()
regr.fit(new_X_train[["DepDelay", "ArrDelay"]], new_X_train["CarrierDelay"])

LinearRegression()

In [16]:
regr.coef_

array([ 0.36813702, -0.06879773])

In [17]:
valores_imputacion = new_X[np.isnan(X.CarrierDelay)]
valores_imputacion = valores_imputacion[["DepDelay", "ArrDelay"]]

In [18]:
nuevos_valores = regr.predict(valores_imputacion)

In [21]:
X.loc[np.isnan(X.CarrierDelay), "CarrierDelay"] = nuevos_valores

In [22]:
X

,AirTime,Distance,TaxiIn,TaxiOut,DepDelay,ArrDelay,CarrierDelay
6418345,174.0,1041,3.0,44.0,-3.0,26.0,0.000000
4739911,161.0,1197,4.0,11.0,14.0,5.0,7.029204
477653,281.0,2586,5.0,23.0,-7.0,-18.0,0.880674
21581,31.0,192,3.0,10.0,-2.0,-3.0,1.689393
6702193,61.0,366,6.0,30.0,-2.0,13.0,0.588629
...,...,...,...,...,...,...,...
5848324,194.0,1438,12.0,17.0,-8.0,5.0,-1.069811
292334,43.0,279,3.0,27.0,44.0,30.0,0.000000
2594354,130.0,916,5.0,13.0,-3.0,-5.0,1.458851
4037582,91.0,612,6.0,15.0,5.0,-3.0,4.266352
